In [1]:
import pandas as pd
import os
import re

In [2]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Padronizacao_conversao'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [3]:
PASTA_DE_DADOS = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_download']
PASTA_SAIDA = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_convertido']
ARQUIVO_OPERADORAS = PARAMETROS_GERAIS['arquivo_operadoras']
ANO_INICIO_ANALISES = int(PARAMETROS_GERAIS['ano_inicio_analises'])

In [4]:
ARQUIVO_OPERADORAS

'http://ftp.dadosabertos.ans.gov.br/FTP/PDA/operadoras_de_plano_de_saude_ativas/Relatorio_cadop.csv'

In [5]:
df_operadoras = pd.read_csv(ARQUIVO_OPERADORAS,dtype=str,sep=";",encoding="ISO-8859-1")

In [6]:
df_operadoras = df_operadoras[df_operadoras['Modalidade'] == 'Cooperativa Médica']

In [7]:
patternPeriodo = re.compile('(.*)T(.*).csv')

In [8]:
lista_arquivos = os.listdir(PASTA_DE_DADOS)

In [9]:
lista_arquivos

['1T2016.csv',
 '3T2014.csv',
 '4T2013.csv',
 '2T2018.csv',
 '1T2019.csv',
 '2T2016.csv',
 '1T2015.csv',
 '1T2018.csv',
 '1T2013.csv',
 '4T2014.csv',
 '2T2014.csv',
 '4T2017.csv',
 '2T2019.csv',
 '4T2015.csv',
 '3T2019.csv',
 '4T2018.csv',
 '2T2017.csv',
 '3T2016.csv',
 '1T2014.csv',
 '3T2013.csv',
 '2T2015.csv',
 '3T2015.csv',
 '2T2013.csv',
 '1T2017.csv',
 '3T2018.csv',
 '3T2017.csv',
 '4T2016.csv']

In [10]:
if (not(os.path.exists(PASTA_SAIDA))):
    os.mkdir(PASTA_SAIDA)

In [11]:
for _idx,arquivo in enumerate(lista_arquivos):
    _match = patternPeriodo.findall(arquivo)[0]
    _trimestre = _match[0]
    _ano =  _match[1]
    print(_ano,_trimestre)
    if (int(_ano) >= ANO_INICIO_ANALISES):
#         print("Inicio")
        df = pd.read_csv(PASTA_DE_DADOS+arquivo,sep=";",dtype=str,encoding="ISO-8859-1")
#         print("DF lido",len(df))
        df = df.merge(df_operadoras,left_on='REG_ANS', right_on='Registro_ANS')
#         print("DF merge",len(df))
        df.insert(0,'TRIMESTRE',_trimestre)
        df.insert(0,'ANO',_ano)
        df.insert(0,'PERIODO',_ano+_trimestre.rjust(2, '0'))
        df.insert(0,'CHAVE',df['CD_CONTA_CONTABIL']+"_"+df['REG_ANS']+"_"+_ano+"_"+_trimestre)
#         print("DF inserts",len(df))
        df['VL_SALDO_FINAL'] = df['VL_SALDO_FINAL'].str.replace(",",".").astype(float)
#         print("DF saldofinal",len(df))
        df.drop(columns=["DATA"],inplace=True)
#         print("DF drop data",len(df))
        df.set_index('CHAVE',inplace=True)
        df.to_csv(PASTA_SAIDA+arquivo,encoding="ISO-8859-1")
    print("Finalizado ",(_idx+1),"de",len(lista_arquivos))

2016 1
Finalizado  1 de 27
2014 3
Finalizado  2 de 27
2013 4
Finalizado  3 de 27
2018 2
Finalizado  4 de 27
2019 1
Finalizado  5 de 27
2016 2
Finalizado  6 de 27
2015 1
Finalizado  7 de 27
2018 1
Finalizado  8 de 27
2013 1
Finalizado  9 de 27
2014 4
Finalizado  10 de 27
2014 2
Finalizado  11 de 27
2017 4
Finalizado  12 de 27
2019 2
Finalizado  13 de 27
2015 4
Finalizado  14 de 27
2019 3
Finalizado  15 de 27
2018 4
Finalizado  16 de 27
2017 2
Finalizado  17 de 27
2016 3
Finalizado  18 de 27
2014 1
Finalizado  19 de 27
2013 3
Finalizado  20 de 27
2015 2
Finalizado  21 de 27
2015 3
Finalizado  22 de 27
2013 2
Finalizado  23 de 27
2017 1
Finalizado  24 de 27
2018 3
Finalizado  25 de 27
2017 3
Finalizado  26 de 27
2016 4
Finalizado  27 de 27
